# AI Red Teaming Agent Local Quickstart (Preview)

This notebook follows the **Run AI Red Teaming Agent locally (preview)** guidance published on 09/18/2025. It shows how to install the preview dependencies, connect to your Azure AI Foundry project, and execute a local scan against a simple callback target.

> ⚠️ Items marked *preview* are provided without an SLA and are not recommended for production workloads.


## Prerequisites

- Python 3.10, 3.11, or 3.12 (PyRIT is not compatible with Python 3.9).
- The `azure-ai-evaluation[redteam]` preview package.
- The Azure CLI authenticated to the tenant that owns your Azure AI Foundry project (`az login`).
- Environment variables that describe your Azure AI Foundry project:
  - Either `AZURE_AI_PROJECT` pointing to the project endpoint URL,
  - **Or** `AZURE_SUBSCRIPTION_ID`, `AZURE_RESOURCE_GROUP`, and `AZURE_PROJECT_NAME`.
- Optional: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_KEY`, and `AZURE_OPENAI_DEPLOYMENT` if you want to scan an Azure OpenAI model configuration later.


In [ ]:
import sys

print(f'Using Python {sys.version}')
if sys.version_info < (3, 10):
    raise RuntimeError('PyRIT requires Python 3.10 or later.')


In [ ]:
# Install or upgrade the Azure AI Evaluation SDK with red teaming extras
%pip install --upgrade "azure-ai-evaluation[redteam]"


## Configure Azure AI project details

The AI Red Teaming Agent stores scan telemetry in your Azure AI Foundry project. Fill in the environment variables before running the next cell. If `AZURE_AI_PROJECT` is defined, it takes precedence over the individual subscription/resource group/project variables.


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from pprint import pprint

project_endpoint = os.environ.get('AIPROJECT_CONNECTION_STRING')

if project_endpoint:
    azure_ai_project = project_endpoint
    print('Using Azure AI project endpoint:')
    print(azure_ai_project)
else:
    required_vars = ['AZURE_SUBSCRIPTION_ID', 'AZURE_RESOURCE_GROUP', 'AZURE_PROJECT_NAME']
    missing = [var for var in required_vars if not os.environ.get(var)]
    if missing:
        raise RuntimeError(
            'Missing required environment variables: ' + ', '.join(missing) +
            '. Set them or define AZURE_AI_PROJECT before continuing.'
        )
    azure_ai_project = {
        'subscription_id': os.environ['AZURE_SUBSCRIPTION_ID'],
        'resource_group_name': os.environ['AZURE_RESOURCE_GROUP'],
        'project_name': os.environ['AZURE_PROJECT_NAME'],
    }
    print('Using Azure AI project configuration:')
    pprint(azure_ai_project)


## Instantiate the AI Red Teaming Agent

`DefaultAzureCredential` will reuse the Azure CLI sign-in, managed identity, or other local credentials supported by the Azure Identity library.


In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
)

print('Red Team agent created. Ready to run scans.')


## Define a simple callback target

This function simulates an application endpoint that always refuses harmful content. The agent will probe it with generated attack prompts.


In [ ]:
def simple_callback(query: str) -> str:
    """Return a templated refusal response for every prompt."""
    return (
        "I'm an AI assistant that follows ethical guidelines. "
        "I cannot provide harmful content."
    )


## Run a baseline red teaming scan

The default agent covers 10 objectives across the four built-in risk categories (violence, hate & unfairness, sexual, and self-harm). The `output_path` captures a JSON scorecard you can review or upload to compliance tooling.


In [ ]:
%%capture --no-display red_team_logs
red_team_result = await red_team_agent.scan(
    target=simple_callback,
    scan_name='Local-Simple-Callback-Scan',
    output_path='My-First-RedTeam-Scan.json',
)


## Inspect scan outputs

Display a condensed view of the scorecard along with the path to the locally saved JSON file.


In [ ]:
import json
from pathlib import Path

output_path = Path('My-First-RedTeam-Scan.json')
if not output_path.exists():
    raise FileNotFoundError('Expected output file was not generated.')

with output_path.open() as f:
    report = json.load(f)

print(f'Read report from: {output_path.resolve()}')
print('Available top-level keys:', list(report.keys()))

risk_summary = report.get('redteaming_scorecard', {}).get('risk_category_summary', [])
attack_summary = report.get('redteaming_scorecard', {}).get('attack_technique_summary', [])

print('Risk category summary:')
print(json.dumps(risk_summary, indent=2))

print('Attack technique summary:')
print(json.dumps(attack_summary, indent=2))


## Optional: Customize risk categories and attack strategies

Uncomment and adjust the following cell to narrow the scan to specific harms or to explore different attack strategies, mirroring the options described in the preview documentation.


In [ ]:
# Example configuration: target two risk categories with five attack objectives each
# and apply a mix of grouped and explicit strategies.
#
# customized_agent = RedTeam(
#     azure_ai_project=azure_ai_project,
#     credential=credential,
#     risk_categories=[
#         RiskCategory.Violence,
#         RiskCategory.HateUnfairness,
#     ],
#     num_objectives=5,
# )
#
# custom_result = await customized_agent.scan(
#     target=simple_callback,
#     scan_name='Customized-Strategy-Scan',
#     attack_strategies=[
#         AttackStrategy.EASY,
#         AttackStrategy.MODERATE,
#         AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),
#     ],
#     output_path='Customized-Strategy-Scan.json',
# )
# print('Customized scan complete.')
